<a href="https://colab.research.google.com/github/KieranJP/Neural-Network-Binary-Classifier/blob/master/Deep_Neural_Network_Binary_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This allows me to upload the csv files used as the training and the test data fom my PC:**

In [0]:
#Uploading Test Data
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot

#Imports Required
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

#Selecting the Csv to use then Displaying it
import pandas as pd
dataset = pd.read_csv("BalancedELFF.csv")

#Using sklearn's split function to split the data into training and testing data.
from sklearn.model_selection import train_test_split

X = dataset.values
Y = dataset.Defective.values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3)

class_names = ['False', 'True']

In [0]:
import keras as K

#Creates a NN
model = Sequential()
#Input Layer
model.add(K.layers.Dense(units=40, input_dim=40, activation='relu', kernel_initializer='truncated_normal')) 
#Hidden Layers
model.add(K.layers.Dense(units=32, activation='relu', kernel_initializer='truncated_normal')) 
model.add(K.layers.Dense(units=32, activation='relu', kernel_initializer='truncated_normal'))
#Output Layers
model.add(K.layers.Dense(units=1, activation='sigmoid', kernel_initializer='truncated_normal'))

simple_sgd = K.optimizers.Adam(lr=0.01)  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [0]:
#Find out the size of each class so can change weighting
trueWeight = len(Y_train[Y_train==False])
falseWeight = len(Y_train[Y_train==True])

history = model.fit(X_train, Y_train, batch_size=32, epochs=250, validation_split=0.33)

In [0]:
np.set_printoptions(precision=4, suppress=True)
eval_results = model.evaluate(X_train, Y_train) 
print("\nLoss, accuracy on test data: ")
print("%0.4f %0.2f%%" % (eval_results[0], \
  eval_results[1]*100))

In [0]:
from sklearn.metrics import classification_report, confusion_matrix 
Y_predict = model.predict(X_test)
Y_predict =(Y_predict>0.5)
print(confusion_matrix(Y_test, Y_predict))
print(classification_report(Y_test, Y_predict))

In [0]:
#I Guess this shows that the model is decently fit as both the training and the test data both have good accuracy?
pyplot.plot(history.history['acc'])
pyplot.plot(history.history['val_acc'])
pyplot.title('Model Train vs Test Accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='upper left')
pyplot.show()

pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('Model Train vs Test loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'test'], loc='upper left')
pyplot.show()